# Algoritmos de U-W-Z para G-S-U.

Esta hoja contiene los resultados de la implementación de los algoritmos del artículo (Urbaniak, Weismantel, Ziegler 1997). De momento no podéis ejecutarla, pero sí podéis leerla, para ver el tipo de cosas que ya podemos calcular. 

El núcleo de lo que he hecho es un programa en SAGE de unas 200 líneas. Cuando nos veamos después de pascua, os diré cómo tenéis que llamarlo desde SAGE.




In [3]:
#%load_ext sage

In [4]:
%run ipproblem

Un problema de IP se declara de la siguiente forma:

In [5]:
P1=IPProblem(c=[1,2,1],
            A=[[1,0,0],[0,0,1]], 
            b=[10,10] )

O bien, simplemente como

In [6]:
P1=IPProblem([1,2,1],
            [[1,0,0],[0,0,1]], 
            [10,10] )

Para comprobar los datos que hemos metido, podemos "imprimir" el problema

In [7]:
print P1

Integer programming problem
    max\{c.x : A x <= b,  0<=x<=u\}
with
c=(1, 2, 1),
A=
[1 0 0]
[0 0 1],
b=(10, 10),
u=(10, 0, 10).


Algunas funciones útiles:

* `is_feasible(v)` determina si el vector v es “feasible”.
* `order(v,w)` determina si los vectores están ordenados para el orden $\prec_c$  del artículo
* `pm_split(v)` devuelve un par de vectores, correspondientes a $v^+$ y $v^-$ del artículo.
* `succ(v)` devuelve el vector $v^\prec$ del artículo

Hay más funciones, pero no voy a dar ejemplos de todas. 

In [8]:
P1.is_feasible(vector(ZZ,[0,0,0]))

True

In [9]:
P1.order(vector(ZZ,[0,2,0]),vector(ZZ,[13,0,1]))

-1

Para las comprobaciones, uso el IP del Ejemplo 2.3 del artículo

In [10]:
P=IPProblem(c=[1,3,2], A=[[1,2,3]], b=[3], u=[1,1,1])
print P

Integer programming problem
    max\{c.x : A x <= b,  0<=x<=u\}
with
c=(1, 3, 2),
A=
[1 2 3],
b=(3),
u=(1, 1, 1).


Cálculo de un test set, simple y llan0; lo hago para los dos problemas que hemos definido, `P1` y `P`.  `test_set` usa el algoritmo 2.1

In [11]:
set_verbose(0)
P.test_set()

[(1, 0, 0),
 (0, 1, 0),
 (0, 0, 1),
 (-1, 1, 0),
 (-1, 0, 1),
 (0, 1, -1),
 (1, 1, -1)]

In [12]:
T=P1.test_set()
T

[(1, 0, 0),
 (0, 0, 1),
 (1, 0, -1),
 (-1, 0, 2),
 (2, 0, -2),
 (-2, 0, 3),
 (-3, 0, 4),
 (3, 0, -3),
 (-4, 0, 5),
 (4, 0, -4),
 (-5, 0, 6),
 (-6, 0, 7),
 (5, 0, -5),
 (-7, 0, 8),
 (-8, 0, 9),
 (6, 0, -6),
 (-9, 0, 10),
 (7, 0, -7),
 (8, 0, -8),
 (9, 0, -9),
 (10, 0, -10)]

Como se puede ver, sobran vectores. Hay que definir varias funciones de reducción, pero no me entretengo. Sólo un ejemplo:

In [13]:
P1.can_reduce_by(v=vector(ZZ, [1,0,-1]), w=vector([10,0,-10]))

True

Ahora podemos reducir el test set que hemos encontrado.

In [14]:
set_verbose(0) # or 2
P1.inter_reduce(T)

[(1, 0, 0), (0, 0, 1)]

La función `minimal_test_set` es una implementación del algoritmo 2.7. Devuelve el único test-set minimal.

In [15]:
P.minimal_test_set()

[(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 1, 0), (-1, 0, 1), (0, 1, -1)]

In [16]:
P1.minimal_test_set()

[(1, 0, 0), (0, 0, 1)]

Por último, `reduced_test_set` primero calcula un test set, sin reducir, y luego aplica reducción. Debe dar lo mismo que antes; el único test-set minimal.

In [17]:
set_verbose(0)
P.reduced_test_set()

[(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 1, 0), (-1, 0, 1), (0, 1, -1)]

In [18]:
P1.reduced_test_set()

[(1, 0, 0), (0, 0, 1)]

Algoritmo 2.10. Mejor que el Grobner simple, pero no necesariamente reducido.

In [19]:
set_verbose(0)
P1.test_set_non_reducibles()

[(1, 0, 0),
 (0, 0, 1),
 (1, 0, -1),
 (-1, 0, 2),
 (2, 0, -2),
 (-2, 0, 3),
 (3, 0, -3),
 (-3, 0, 4),
 (4, 0, -4),
 (-4, 0, 5),
 (5, 0, -5),
 (-5, 0, 6),
 (6, 0, -6),
 (-6, 0, 7),
 (7, 0, -7),
 (-7, 0, 8),
 (8, 0, -8),
 (-8, 0, 9),
 (9, 0, -9),
 (-9, 0, 10),
 (10, 0, -10)]

In [20]:
P.test_set_non_reducibles()

[(1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 1, 0), (-1, 0, 1), (0, 1, -1)]

Anulo los cómputos de tiempo hasta que se estabilicen los algoritmos.

In [21]:
#%timeit -n100 P.groebner_simple()

In [22]:
#%timeit -n100 P.groebner_with_reduction()

In [23]:
#%timeit -n100 P.groebner_reduced()

In [24]:
#%timeit -n100 P.groebner_non_reducibles()

Cálculo del óptimo a partir de un factible. Siempre se usa el test set minimal.


In [25]:
%run ipproblem
P=IPProblem(c=[1,3,2], A=[[1,2,3]], b=[3], u=[1,1,1])
set_verbose(0)
P.walk_to_best([0,0,0])


(1, 1, 0)

También podemos pedir el camino completo desde el inicial al final.

In [26]:
P.walk_to_best([0,0,0], path=True)

[(0, 0, 0), (0, 1, 0), (1, 1, 0)]

In [27]:
P.walk_to_best([0,0,1])

(1, 1, 0)

In [28]:
P.walk_to_best([10,10,10]) # not feasible

False

In [29]:
P1.walk_to_best([0,0,0], path=True)

[(0, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (5, 0, 0),
 (6, 0, 0),
 (7, 0, 0),
 (8, 0, 0),
 (9, 0, 0),
 (10, 0, 0),
 (10, 0, 1),
 (10, 0, 2),
 (10, 0, 3),
 (10, 0, 4),
 (10, 0, 5),
 (10, 0, 6),
 (10, 0, 7),
 (10, 0, 8),
 (10, 0, 9),
 (10, 0, 10)]